In [ ]:
#Revised September 2019 - remove plotting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
#from mpl_toolkits.basemap import Basemap
import datetime
from scipy import signal

import matplotlib.dates as mdates
import glob, os
import math
from matplotlib.backends.backend_pdf import PdfPages
import time

In [ ]:
%matplotlib inline
plt.ion()

In [ ]:
#full list
log_files=[
['/home/pkotze/python/SKA-MPI Power measurements/power_test_A1_1/CSV/97intvl.csv','Initial A power check',[
    ['A1.1',pd.Timestamp('09:12:40')],
    ['Mains CB',pd.Timestamp('09:12:55')],
    ['A1.2',pd.Timestamp('09:13:49')],
    ['Controller',pd.Timestamp('09:13:55')],
    ['A1.3',pd.Timestamp('09:15:35')],
    ['A1.4 DC fan',pd.Timestamp('09:16:05')],
    ['A1.4 PSC?',pd.Timestamp('09:16:43')],
    ['A1.5',pd.Timestamp('09:17:28')],
    ['V.M Consumption',pd.Timestamp('09:18:33')]]],
['/home/pkotze/python/SKA-MPI Power measurements/power_test_A1_2/CSV/97intvl.csv','Repeat A power check',[
    ['A1.1',pd.Timestamp('10:12:54')],
    ['A1.2',pd.Timestamp('10:12:55')],
    ['A1.3',pd.Timestamp('10:13:35')],
    ['A1.4',pd.Timestamp('10:14:10')],
    ['A1.5',pd.Timestamp('10:15:38')],
    ['A1.6',pd.Timestamp('10:16:20')],
    ['A1.7',pd.Timestamp('10:16:35')],
    ['A1.8',pd.Timestamp('10:16:55')],
    ['A1.9',pd.Timestamp('10:17:20')]]],
['/home/pkotze/python/SKA-MPI Power measurements/power_test_C 20elev/CSV/97intvl.csv','C power check 20 elev'],
['/home/pkotze/python/SKA-MPI Power measurements/power_test_C 85elev/CSV/97intvl.csv','C power check 85 elev'],
['/home/pkotze/python/SKA-MPI Power measurements/power_test_D/CSV/97intvl.csv','D  power check']]

In [ ]:
log_files[0][0]


In [ ]:
#Events 
#  recorded 20ms before detection + 200ms upon detection + 30ms after detection is 250ms
# 2558 samples per file => 100us interval?!

#Inrush current fl uctuations are captured in RMS at a fast 10ms sampling rate and displayed across a 30-second window

#def event_plotter(log_file):
#log_file=event_file[2]
def event_plotter(log_file):
    print(log_file)
    df=pd.read_csv(log_file, nrows=1, names=['START_TIME'],parse_dates=['START_TIME'])
    event_start=(df['START_TIME'][0])
    date_rng = pd.date_range(start=event_start, periods=2558, freq='100us')

    df=pd.read_csv(log_file, header=1, na_values=' ----- ')
    #print(df)
    #date_rng = pd.date_range(start=event_start, periods=2558, freq='100us')
    #print(type(date_rng))

    u1=np.asarray( df['U1'] ).astype(float)
    u2=np.asarray( df['U2'] ).astype(float)
    u3=np.asarray( df['U3'] ).astype(float)
    i1=np.asarray( df['I1'] ).astype(float)
    i2=np.asarray( df['I2'] ).astype(float)
    i3=np.asarray( df['I3'] ).astype(float)
    i4=np.asarray( df['I4'] ).astype(float)

    fig,[ax1, ax2] = plt.subplots(nrows=2, ncols=1, figsize=(20,15))

    ax1.set(title=log_file + " => " + str(event_start), ylabel='Phase voltage [V]') 
    xlabel='Timestep [units]'
    ax1.plot(date_rng, u1)
    ax1.plot(date_rng, u2)
    ax1.plot(date_rng, u3)
    ymin=-350
    ymax=350 #253 max?
    ystep=50
    #ymin=int(min([min(u1),min(u2),min(u3)])*1.25)
    #ymax=int(max([max(u1),max(u2),max(u3)])*1.25)
    ax1.set_yticks(range(ymin,ymax,ystep))
    ax1.set_ylim(ymin,ymax)
    ax1.xaxis.set_major_locator(mdates.MicrosecondLocator(interval=20000))
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H %M %S.%f')) 
    ax1.tick_params(axis='x', labelrotation=35)
    #_=plt.xticks(rotation=65)
    ax1.legend(['U1 [V]','U2 [V]','U3 [V]'])
    ax1.grid()

    ax2.set(ylabel='Phase current [I]') 
    ax2.plot(date_rng, i1)
    ax2.plot(date_rng, i2)
    ax2.plot(date_rng, i3)
    ax2.plot(date_rng, i4)
    ymin=-50
    ymax=50
    ystep=1
    ymin=math.floor(min([min(i1),min(i2),min(i3),min(i4)])*1.25)
    ymax=math.ceil(max([max(i1),max(i2),max(i3),max(i4)])*1.25)
    ystep=round((ymax-ymin)//10.0,1)
    print(ystep)
    #ax2.set_yticks(range(ymin,ymax,ystep))
    ax2.set_ylim(ymin,ymax)
    ax2.xaxis.set_major_locator(mdates.MicrosecondLocator(interval=20000))
    ax2.xaxis.set_major_formatter(mdates.DateFormatter('%H %M %S.%f')) 
    ax2.tick_params(axis='x', labelrotation=35)
    ax2.legend(['I1 [A]','I2 [A]','I3 [A]','I4 [A]'])
    ax2.grid()
    fig.savefig(log_file + '_event1.png')


In [ ]:
last=log_files[0][0].rfind('/')
log_files[0][0][0:last+1]

In [ ]:
#process all lines in log_files

for line in log_files:
    mydir=line[0][0:line[0].rfind('/')+1]
    description=line[1]
    print(description, mydir)
    event_file=[file for file in glob.glob(mydir + "97evt*.csv")]
    event_file.sort()
    print(event_file)
    with PdfPages(mydir + '_events_' + description + '_' + str(int(time.time())) +'.pdf') as pdf:
        for file in event_file:
            event_plotter(file)
            pdf.savefig()  # saves the current figure into a pdf page
            plt.close()

In [ ]:
#Test A1_2
mydir='/home/pkotze/python/SKA-MPI Power measurements/power_test_A1_1/CSV/'

event_file=[file for file in glob.glob(mydir + "97evt*.csv")]
event_file.sort()

#event_file.remove('97evt008.csv')

print(event_file)
with PdfPages(file+'_events_' + str(int(time.time())) +'.pdf') as pdf:
    for file in event_file:
        event_plotter(file)
        pdf.savefig()  # saves the current figure into a pdf page

In [ ]:
#A1_2
mydir='/home/pkotze/python/SKA-MPI Power measurements/power_test_A1_2/CSV/'

event_file=[file for file in glob.glob(mydir + "97evt*.csv")]
event_file.sort()

#event_file.remove('97evt008.csv')

print(event_file)
for file in event_file:
    event_plotter(file)

In [ ]:
#B or C
mydir='/home/pkotze/python/SKA-MPI Power measurements/power_test_C 20elev/CSV/'

event_file=[file for file in glob.glob(mydir + "97evt*.csv")]
event_file.sort()

#event_file.remove('97evt008.csv')

print(event_file)
for file in event_file:
    event_plotter(file)

In [ ]:
#B or C
mydir='/home/pkotze/python/SKA-MPI Power measurements/power_test_C 85elev/CSV/'

event_file=[file for file in glob.glob(mydir + "97evt*.csv")]
event_file.sort()

#event_file.remove('97evt008.csv')

print(event_file)
for file in event_file:
    event_plotter(file)

In [ ]:
#D
mydir='/home/pkotze/python/SKA-MPI Power measurements/power_test_D/CSV/'

event_file=[file for file in glob.glob(mydir + "97evt*.csv")]
event_file.sort()

#event_file.remove('97evt008.csv')

print(event_file)
for file in event_file:
    event_plotter(file)

In [ ]:
# DEVELOPMENT CELL
df=pd.read_csv(log_file, nrows=1, names=['START_TIME'],parse_dates=['START_TIME'])
event_start=(df['START_TIME'][0])
print(type(event_start))

In [ ]:
# DEVELOPMENT CELL
date_rng = pd.date_range(start=event_start, periods=2558, freq='100us')
'''print(type(date_rng))
#date_rng.to_datetime()
print(date_rng.shape)
print(date_rng)
plt.plot(date_rng)
'''

In [ ]:
# DEVELOPMENT CELL

log_file=event_file[9]
#def event_plotter(log_file):
print(log_file)
df=pd.read_csv(log_file, nrows=1, names=['START_TIME'],parse_dates=['START_TIME'])
event_start=(df['START_TIME'][0])
date_rng = pd.date_range(start=event_start, periods=2558, freq='100us')

df=pd.read_csv(log_file, header=1, na_values=' ----- ')
#print(df)
#date_rng = pd.date_range(start=event_start, periods=2558, freq='100us')
#print(type(date_rng))

u1=np.asarray( df['U1'] ).astype(float)
u2=np.asarray( df['U2'] ).astype(float)
u3=np.asarray( df['U3'] ).astype(float)
i1=np.asarray( df['I1'] ).astype(float)
i2=np.asarray( df['I2'] ).astype(float)
i3=np.asarray( df['I3'] ).astype(float)
i4=np.asarray( df['I4'] ).astype(float)

fig,[ax1, ax2] = plt.subplots(nrows=2, ncols=1, figsize=(20,15))

ax1.set(title=log_file + " => " + str(event_start), ylabel='Phase voltage [V]') 
xlabel='Timestep [units]'
ax1.plot(date_rng, u1)
ax1.plot(date_rng, u2)
ax1.plot(date_rng, u3)
ymin=-350
ymax=350 #253 max?
ystep=50
ax1.set_yticks(range(ymin,ymax,ystep))
ax1.set_ylim(ymin,ymax)
ax1.xaxis.set_major_locator(mdates.MicrosecondLocator(interval=20000))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%S.%f')) 
ax1.tick_params(axis='x', labelrotation=35)
#_=plt.xticks(rotation=65)
ax1.legend(['U1 [V]','U2 [V]','U3 [V]'])
ax1.grid()

ax2.set(title=log_file + " => " + str(event_start), ylabel='Phase current [I]') 
ax2.plot(date_rng, i1)
ax2.plot(date_rng, i2)
ax2.plot(date_rng, i3)
ax2.plot(date_rng, i4)
ymin=-50
ymax=50
ystep=5
ax2.set_yticks(range(ymin,ymax,ystep))
ax2.set_ylim(ymin,ymax)
ax2.xaxis.set_major_locator(mdates.MicrosecondLocator(interval=20000))
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%S.%f')) 
ax1.tick_params(axis='x', labelrotation=35)
ax2.legend(['I1 [A]','I2 [A]','I3 [A]','I4 [A]'])
ax2.grid()

Check timestamps step size to see if data average/interval was changed.